<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/QuantLib/05-OIS-QuantLib-UserCurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example to show how to use the AADC with QuantLib library in Python. It's based on OIS-QuantLib-example.ipynb so please check it first.

In this example we define custom curve interpolation in Python deriving from C++ abstract class and using object created on Python size do perform pricing on C++ side. We than demostrate that AADC recording is fully functioning in this use case.

In [1]:
import sys
!pip install https://matlogica.com/DemoReleases/aadcquantlib-1.7.5.30-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

  Using cached https://matlogica.com/DemoReleases/aadcquantlib-1.7.5.27-cp311-cp311-linux_x86_64.whl (38.0 MB)


In [2]:
import aadc
import aadc.quantlib as ql
import numpy as np

Initialize QuantLib as normal

In [3]:
today = ql.Date(19, ql.October, 2020)
ql.Settings().evaluationDate = today

In [4]:
# create the yield curve zero rates using native double arrays
dates = [    ql.Date(19,10,2020),    ql.Date(19,11,2020),    ql.Date(19, 1,2021),    ql.Date(19, 4,2021),    ql.Date(19,10,2021),    ql.Date(19, 4,2022),    ql.Date(19,10,2022),    ql.Date(19,10,2023),    ql.Date(19,10,2025),    ql.Date(19,10,2030),    ql.Date(19,10,2035),    ql.Date(19,10,2040),]

forecast_rates = [    -0.004,    -0.002,    0.001,    0.005,    0.009,    0.010,    0.010,    0.012,    0.017,    0.019,    0.028,    0.032]


In [5]:
discount_spread = - 0.001
MyCurveOutputDebug = True

In [6]:
# ql.YieldTermStructure is interface class defined on QuantLib side
# We override discountImpl() method to define custom discount factor interpolation method

class MyCurve(ql.YieldTermStructure):
    def __init__(self, dates, rates):
        super(MyCurve, self).__init__(ql.Actual365Fixed())
        self.dates = dates
        self.rates = rates
    def discountImpl(self, t):
        # strange custom method to interpolate, but lets see if it works.
        if MyCurveOutputDebug:
            print("C++ called Python for discount factor at ",t)
        # t is converted to float to show that it's not "stochastic"
        return np.exp(-(self.rates[0] + self.rates[1]) * float(t))
    def maxDate(self):
        return ql.Date(31,12,2100)

In [7]:
# Define function to price single OIS swap. Note that discount_rates are computed on Python side, but pricing is done on C++ side.
# We calculate risk w.r.t forecast rates and discount spread. AAD tape is used to compute sensitivities on python and C++ side.

def PriceOIS(dates, forecast_rates, discount_spread):
    forecast_curve = ql.ZeroCurve(dates, forecast_rates, ql.Actual365Fixed())
    
    discount_rates = [ r + discount_spread for r in forecast_rates ]

    # This time discount curve implemented on Python side
    my_discount_curve = MyCurve(dates, discount_rates)

    forecast_handle = ql.YieldTermStructureHandle(forecast_curve)
    discount_handle = ql.YieldTermStructureHandle(my_discount_curve)

    swap = ql.MakeOIS(swapTenor=ql.Period(5, ql.Years),
                    overnightIndex=ql.Eonia(forecast_handle),
                    fixedRate=0.002)

    swapEngine = ql.DiscountingSwapEngine(discount_handle)
    swap.setPricingEngine(swapEngine)

    return swap.NPV()

In [8]:
# Price the swap using regular call to QuantLib with native double types
print(PriceOIS(dates, forecast_rates, discount_spread))

C++ called Python for discount factor at  idouble(0.00e+00)
C++ called Python for discount factor at  idouble(1.22e+02)
C++ called Python for discount factor at  idouble(1.23e+02)
C++ called Python for discount factor at  idouble(1.24e+02)
C++ called Python for discount factor at  idouble(1.25e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.22e+02)
C++ called Python for discount factor at  idouble(1.23e+02)
C++ called Python for discount factor at  idouble(1.24e+02)
C++ called Python for discount factor at  idouble(1.25e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at

In [9]:
# Create object to hold AADC Kernel recording
funcs = aadc.Functions()

In [10]:
# Start recording. All aadc.idouble() operations will be recorded
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


In [11]:
# convert each element in rates and discount_spread to idouble
forecast_rates = [aadc.idouble(r) for r in forecast_rates]
discount_spread = aadc.idouble(discount_spread)
forecast_rates

[idouble(-4.00e-03),
 idouble(-2.00e-03),
 idouble(1.00e-03),
 idouble(5.00e-03),
 idouble(9.00e-03),
 idouble(1.00e-02),
 idouble(1.00e-02),
 idouble(1.20e-02),
 idouble(1.70e-02),
 idouble(1.90e-02),
 idouble(2.80e-02),
 idouble(3.20e-02)]

In [12]:
# mark each idouble as input and save the reference id
forecast_ratesArgs = [r.mark_as_input() for r in forecast_rates]
discount_spreadArg = discount_spread.mark_as_input()
forecast_ratesArgs

[Arg(6),
 Arg(7),
 Arg(8),
 Arg(9),
 Arg(10),
 Arg(11),
 Arg(12),
 Arg(13),
 Arg(14),
 Arg(15),
 Arg(16),
 Arg(17)]

In [13]:
# Call the PriceOIS function with the idouble arguments and record the operations
swap_NPV = PriceOIS(dates, forecast_rates, discount_spread)

C++ called Python for discount factor at  idouble(0.00e+00)
C++ called Python for discount factor at  idouble(1.22e+02)
C++ called Python for discount factor at  idouble(1.23e+02)
C++ called Python for discount factor at  idouble(1.24e+02)
C++ called Python for discount factor at  idouble(1.25e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.22e+02)
C++ called Python for discount factor at  idouble(1.23e+02)
C++ called Python for discount factor at  idouble(1.24e+02)
C++ called Python for discount factor at  idouble(1.25e+02)
C++ called Python for discount factor at  idouble(1.26e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at  idouble(1.21e+02)
C++ called Python for discount factor at

In [14]:
# Note that swap_NPV is an aadc.idouble() object. [rv] - random variable [adj] - adjoint flag
print(swap_NPV)

idouble([AAD[rv] [adj] :1437,7.81e-02])


In [15]:
# Mark the swap_NPV as output and save the reference
swapNPVRes = swap_NPV.mark_as_output()
swapNPVRes

Res(1437)

In [16]:
# Stop recording
funcs.stop_recording()

In [17]:
# Check if recording is safe to use for arbitrary inputs
# This will print the list of active to passive extract locations
# To use the recording for arbitrary inputs, the active to passive extract locations should be 0
funcs.print_passive_extract_locations()

Number active to passive conversions: 0 while recording Python


'Number active to passive conversions: 0 while recording Python\n'

In [18]:
# swap_NPV is still an aadc.idouble() object after recording
print(swap_NPV)

idouble(7.81e-02)


In [19]:
# But it can be safely converted to a native double
print(swap_NPV+0.0)

0.07811358154792103


In [20]:
# Create new input zero rates and discount spread
# Set vol to 0 to avoid random perturbations and compare bump sensitivities with AADC sensitivities
vol=0.01
num_scenarios=10000

In [21]:
# Basic perturbations to the zero rates and discount spread
inputs = {}
for rArg, r in zip(forecast_ratesArgs, forecast_rates):
    inputs[rArg] = float(r) * np.random.normal(1, vol, num_scenarios) 
inputs[discount_spreadArg] = discount_spread * np.random.normal(1, vol, num_scenarios)
inputs[discount_spreadArg]

array([-0.00099438, -0.00100391, -0.00100806, ..., -0.00100053,
       -0.00100255, -0.00101242])

In [22]:
# Request to AADC evaluation.
# We want swapNPV output and the sensitivities of swapNPV to the zero rates and discount spread
request = {swapNPVRes:forecast_ratesArgs + [ discount_spreadArg ]}

In [23]:
# Run AADC Kernel for array inputs using 4 CPU threads and avx2
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

In [24]:
# AADC returns vector of results for each scenario
print("Result", Res[0][swapNPVRes])

Result [0.07852986 0.07957041 0.07789995 ... 0.07904822 0.07893999 0.07863886]


In [25]:
# Bucketed sensitivities of swapNPV to the zero rates
print("Forecast rates risk : ")
for rArg in forecast_ratesArgs:
    print("dNPV/dR", np.average(Res[1][swapNPVRes][rArg]))

Forecast rates risk : 
dNPV/dR -0.29023890963702526
dNPV/dR -0.28538405338105166
dNPV/dR 0.0
dNPV/dR 0.0
dNPV/dR -0.010100191516789956
dNPV/dR -0.00011222435018655461
dNPV/dR -0.027148903356685326
dNPV/dR -0.07099775795862955
dNPV/dR 5.321477111536543
dNPV/dR 0.005865749901972719
dNPV/dR 0.0
dNPV/dR 0.0


In [26]:
# Sensitivity of swapNPV to the discount spread
print("Discount spread risk : ")
print("dNPV/dR", (Res[1][swapNPVRes][discount_spreadArg]))

Discount spread risk : 
dNPV/dR [-0.5741825  -0.58599549 -0.56807253 ... -0.57985008 -0.5781844
 -0.57472045]


In [27]:
# Compare AADC sensitivities with sensitivities computed by bumping the zero rates
# For perfect match make sure to set vol=0
print("Check with bumping")
MyCurveOutputDebug = False
swap_NPV = PriceOIS(dates, forecast_rates, discount_spread)
print("Forecast rates risk : ")
for i in range(len(forecast_rates)):
    forecast_rates[i] += 0.0001
    swap_NPV_up = PriceOIS(dates, forecast_rates, discount_spread)
    forecast_rates[i] -= 0.0002
    swap_NPV_down = PriceOIS(dates, forecast_rates, discount_spread)
    forecast_rates[i] += 0.0001
    AAD_risk = np.average(Res[1][swapNPVRes][forecast_ratesArgs[i]])
    FD_risk =(swap_NPV_up - swap_NPV_down) / 0.0002 
    print(i, FD_risk, AAD_risk, FD_risk - AAD_risk)

Check with bumping
Forecast rates risk : 
0 -0.2902170888068739 -0.29023890963702526 2.1820830151342285e-05
1 -0.2853622292037922 -0.28538405338105166 2.1824177259488486e-05
2 0.0 0.0 0.0
3 0.0 0.0 0.0
4 -0.010100234759530258 -0.010100191516789956 -4.324274030173547e-08
5 -0.0001122248309559648 -0.00011222435018655461 -4.807694101903676e-10
6 -0.027155112216209476 -0.027148903356685326 -6.208859524150373e-06
7 -0.07098174587036321 -0.07099775795862955 1.601208826633449e-05
8 5.3214702037439325 5.321477111536543 -6.907792610810759e-06
9 0.005865739311633478 0.005865749901972719 -1.0590339240973412e-08
10 0.0 0.0 0.0
11 0.0 0.0 0.0


In [28]:
# Sensitivity of swapNPV to the discount spread
# For perfect match make sure to set vol=0
print("Discount spread risk : ")
discount_spread += 0.0001
swap_NPV_up = PriceOIS(dates, forecast_rates, discount_spread)
discount_spread -= 0.0002
swap_NPV_down = PriceOIS(dates, forecast_rates, discount_spread)
discount_spread += 0.0001
AAD_risk = np.average(Res[1][swapNPVRes][discount_spreadArg])
FD_risk =(swap_NPV_up - swap_NPV_down) / 0.0002
print(FD_risk, AAD_risk, FD_risk - AAD_risk)

Discount spread risk : 
-0.5700052712519221 -0.5700488687982553 4.3597546333207227e-05


In [29]:
import pickle
MyBookedTrade = {
    "Kernel" : funcs,
    "Inputs" : {
        "forecast_rates" : forecast_ratesArgs,
        "discount_spread" : discount_spreadArg
    },
    "Outputs" : {
        "swapNPV" : swapNPVRes
    }
}
with open('05-Kernel.pkl', 'wb') as f:  # open a text file
    pickle.dump(MyBookedTrade, f)